In [36]:
import os
import pandas as pd

In [37]:
def create_phenotypes_count_df(phenotype_files):
    df_list = []
    for file, patient_id in zip(files_list, patient_ids_list):
        print(f"Loading: {file}")
        df = pd.read_csv(file, index_col=0).drop_duplicates(subset='label')
        df['patient_id'] = patient_id
        print(df.head())
        df_list.append(df)
    
    data = pd.concat(df_list, ignore_index=True)
    
    
    # User-specified grouping variables
    grouping_vars = ['patient_id', 'phenotype']
    
    # Metadata columns = all except grouping vars
    # metadata_vars = ['x', 'y', 'area', 'region_num', 'label']
    metadata_vars = []
    
    # Drop duplicates to get one row per patient for metadata
    metadata = data.drop_duplicates(subset=grouping_vars[0])[[grouping_vars[0]] + metadata_vars]
    
    # Count occurrences
    counts = data.groupby(grouping_vars).size().unstack(fill_value=0).reset_index()
    
    # Merge counts with metadata
    counts_data = metadata.merge(counts, on=grouping_vars[0])
    
    clin_data = pd.read_csv('/hpcnfs/scratch/DIMA/chiodin/tests/patients_analysis/data/clinical_data_selected.csv')
    clin_data['patient_id'] = clin_data['patient_id'].astype(str)
    
    counts_data = counts_data.merge(
        clin_data[['patient_id', 'aneuploidy']],
        left_on = 'patient_id',
        right_on='patient_id',
        how='left'
    )
    
    return counts_data

In [38]:
filename = 'phenotypes_20250723.csv'
root_dir = '/hpcnfs/techunits/imaging/work/ATTEND/analysis_runs'

patient_ids_list = [
    '23S64',
    '35138',
    '19E110099A4',
    '1826315',
    'B19-10215',
    '21S188',
    '21-37601',
    '1923575',
    '23S22',
    '1818600'
    ]



files_list = [os.path.join(root_dir, f"{patient_id}_analysis", "output", filename) for patient_id in patient_ids_list]

In [ ]:
counts_data = create_phenotypes_count_df(files_list)

Loading: /hpcnfs/techunits/imaging/work/ATTEND/analysis_runs/23S64_analysis/output/phenotypes_20250723.csv
     ARID1A      CD14     CD163       CD3       CD4      CD45       CD8  \
0 -0.008961 -0.553084 -0.004687  0.132325 -3.925256 -0.353808 -1.875921   
1  1.690660 -1.510505 -1.392588 -5.067700 -3.925256 -3.147156 -1.875921   
2  0.153619 -0.541420 -0.215326  0.425215 -3.925256  0.160855 -1.875921   
3  0.353440 -0.205548 -0.030887  0.280884  0.151096 -0.304736  0.421245   
4  0.001327 -0.695104 -0.204432 -0.290032 -0.207402 -0.393301 -0.493087   

      FOXP3     L1CAM       P53  ...  VIMENTIN         DAPI            x  \
0 -0.568744 -0.129070 -0.371796  ... -1.120854  3169.023499  1975.988251   
1 -1.575011 -2.516095 -1.103221  ... -1.996665  3297.807890   768.566038   
2  0.106746 -0.305112 -0.410416  ...  1.508257  3671.099291  3619.693262   
3  0.391186 -0.190982 -0.327485  ...  1.459564  5558.839286  3484.758929   
4 -0.588815 -2.516095 -0.509986  ... -1.116109  4627.747967  1

In [35]:
# os.chdir('/hpcnfs/scratch/DIMA/chiodin/tests/patients_analysis')

counts_data.to_csv('data/phenotype_counts_20250730.csv', index=False)